In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

In [ ]:
c = MongoClient('localhost', 27017)
db = c.offers

In [ ]:
offers = []
collection = db.offer
for coupons in collection.find():
    offers.append(coupons)

In [ ]:
offers_df = pd.DataFrame(offers)

In [ ]:
offers_df.locationKeywords = offers_df.locationKeywords.apply(lambda y: [{'keyword': 'facet-unknown'}] if len(y)==0 else y)
facet = []
redemption_task = []
categories = []
store_name = []
description = [] 
lat = []
lon = []

for i in range(len(offers_df)):
    facet.append(offers_df.locationKeywords[i][0]['keyword'])
    store_name.append(offers_df.offerStore[i]['name'])
    lat.append(offers_df.offerStore[i]["physicalLocation"]['geolocation']['lat'])
    lon.append(offers_df.offerStore[i]["physicalLocation"]['geolocation']['lon'])
    
    if len(offers_df.redemptionMethods[i]) == 1:
        if "instore_print" in offers_df.redemptionMethods[i]:
            redemption_task.append('hard')
        if "instore" in offers_df.redemptionMethods[i]:
            redemption_task.append('medium')
        if "call" in offers_df.redemptionMethods[i]:
            redemption_task.append('easy')
    elif len(offers_df.redemptionMethods[i]) == 2:
        if "call" not in offers_df.redemptionMethods[i]:
            redemption_task.append('medium')
        else:
            redemption_task.append("easy")
    else:
        redemption_task.append('easy')
    
    try:
        categories.append(offers_df.categories[i][0]['categoryParentName'])
    except KeyError:
        categories.append(offers_df.categories[i][0]['categoryName'])
        
    try:
        description.append(offers_df.offerStore[i]["description"])
    except KeyError:
        description.append("Unavailable")
        
    

In [ ]:
offers_df = offers_df.drop(['_class','categories', 'additionalProperties','facets', 'links', 'startedOn','expiresOn', 'logoUrl', 'offerKey','redemptionMethods', 'offerSet', 'offerGroupKey', 'offerStore', 'locationKeywords', 'nationalOffer'], axis=1)

In [ ]:
offers_df['facet'] = pd.Series(facet, index=offers_df.index)
offers_df['redemption_task'] = pd.Series(redemption_task, index=offers_df.index)
offers_df['categories'] = pd.Series(categories, index=offers_df.index)
offers_df['store_name'] = pd.Series(store_name, index=offers_df.index)
offers_df['description'] = pd.Series(description, index=offers_df.index)
offers_df['latitude'] = pd.Series(lat, index=offers_df.index)
offers_df['longitude'] = pd.Series(lon, index=offers_df.index)

In [ ]:
offers_df

In [ ]:
offers_df.keys()

In [ ]:
offers_text_data = offers_df.drop(['discountType', 'discountValue', 'maximumAward', 'minimumPurchase', 'offerValue', 'promotionCode','savingsAmount', 'usesPerPeriod', 'redemption_task', 'categories', 'latitude', 'longitude'], axis=1)

In [ ]:
a = list(offers_text_data)
a.remove("_id")

offers_numeric_data = offers_df.drop(a, axis=1)

In [ ]:
offers_text_data.head()

In [ ]:
offers_numeric_data.head()